## Setup environment ##
- Open terminal and type
`python3 -m venv venv`
- And to activate this environment
`source venv/bin/activate`
- Just for infomation to deactifate execute `deactiate`

In [3]:
%pip install langchain
%pip install langchain-ollama
%pip install python-dotenv
%pip install openai
%pip install langchain-openai
%pip install langsmith



[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may ne

## Intercting with LLM and setup langsmith##
- go to https://www.langchain.com/ click Products->Observability 
- Try LangSmith
- Create new Project

- Copy "Configure environment" to .env file
- Load .env file

In [4]:
 # MUST be done BEFORE any LangChain imports
import os
from dotenv import load_dotenv

# Load environment first
load_dotenv('./../.env')

# Set LangSmith environment variables BEFORE importing anything from LangChain
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = os.getenv('LANGSMITH_ENDPOINT', 'https://api.smith.langchain.com')
os.environ["LANGSMITH_API_KEY"] = os.getenv('LANGSMITH_API_KEY')
os.environ["LANGSMITH_PROJECT"] = os.getenv('LANGSMITH_PROJECT', 'default')

# Debug: Print to verify values are loaded
print("=== LangSmith Configuration ===")
print(f"Endpoint: {os.environ.get('LANGSMITH_ENDPOINT')}")
print(f"API Key exists: {'Yes' if os.environ.get('LANGSMITH_API_KEY') else 'No'}")
print(f"Project: {os.environ.get('LANGSMITH_PROJECT')}")
print(f"Tracing enabled: {os.environ.get('LANGSMITH_TRACING')}")
print("================================")

# NOW import LangChain components
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain.agents import create_agent
from langchain_core.prompts import ChatPromptTemplate
from langsmith import Client

# Verify LangSmith connection
try:
    client = Client()
    print(f"✅ Connected to LangSmith successfully")
except Exception as e:
    print(f"❌ LangSmith connection failed: {e}")

# Create LLM
llm = ChatOllama(
    base_url="http://localhost:11434",
    model="qwen2.5:latest",
    temperature=0.7,
    max_tokens=1024
)

# Create tool with proper decorator
@tool
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

# Create agent using create_agent (simpler approach)
agent = create_agent(
    model=llm,
    tools=[get_weather],
    system_prompt="You are a helpful assistant"
)

# Run with tracing
print("\n=== Running Agent ===")
result = agent.invoke({"messages": [{"role": "user", "content": "What is the weather in Almere? Tell me a joke about me"}]})
print("Result:", result)

=== LangSmith Configuration ===
Endpoint: https://api.smith.langchain.com
API Key exists: Yes
Project: sudha-langsmith
Tracing enabled: true
✅ Connected to LangSmith successfully

=== Running Agent ===
Result: {'messages': [HumanMessage(content='What is the weather in Almere? Tell me a joke about me', additional_kwargs={}, response_metadata={}, id='2230175f-8d1d-451b-90ea-69f1eb4155cc'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2026-01-31T21:45:57.135201Z', 'done': True, 'done_reason': 'stop', 'total_duration': 21947136066, 'load_duration': 6903334299, 'prompt_eval_count': 157, 'prompt_eval_duration': 11347141917, 'eval_count': 21, 'eval_duration': 3651757691, 'logprobs': None, 'model_name': 'qwen2.5:latest', 'model_provider': 'ollama'}, id='lc_run--019c1604-bcd0-7e70-9254-40943f87e15a-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Almere'}, 'id': 'd48eb6ea-1e97-4717-a351-220cd4a33131', 'type': 'tool_call'}], in

## Prompt and chattemplate ##

In [ ]:
from langchain_core.prompts import PromptTemplate

Prompt_template = PromptTemplate.from_template("What is advantage of running LLM in {place} ")
prompt = Prompt_template.invoke(place="local machine")
print(prompt)
content = llm.invoke(prompt)
print(content)

What is advantage of running LLM in local machine 
content="Running Large Language Models (LLMs) on a local machine can offer several advantages, but it's important to consider the limitations as well. Here are some key benefits:\n\n1. **Control and Privacy**: Running models locally gives you complete control over data handling and processing. This is particularly useful when dealing with sensitive information that should not leave your system.\n\n2. **Customization**: You can tailor the model settings, parameters, or even fine-tune the model to better suit specific needs without relying on external services. This can be beneficial for applications where customization is crucial.\n\n3. **Latency Reduction**: Local execution reduces latency compared to cloud-based solutions because there's no need to transmit data over a network. This can be particularly important in real-time applications or scenarios requiring quick responses.\n\n4. **Cost Efficiency**: For occasional use cases, runni

## Chat Prompt Template

In [8]:
from langchain_core.prompts import ChatPromptTemplate

Prompt_template = ChatPromptTemplate([
    ("system","You are an LLM expert"),
    ("user","What is advantage of running LLM in {place}")
                                       ])
prompt = Prompt_template.format(place="local machine")
print(prompt)
# content = llm.invoke(prompt) or 
for str in llm.stream(prompt):
    print(str)

System: You are an LLM expert
Human: What is advantage of running LLM in local machine
content='Running' additional_kwargs={} response_metadata={} id='lc_run--019c1631-647b-7570-9709-1e7f8f6ef985' tool_calls=[] invalid_tool_calls=[] tool_call_chunks=[]
content=' Large' additional_kwargs={} response_metadata={} id='lc_run--019c1631-647b-7570-9709-1e7f8f6ef985' tool_calls=[] invalid_tool_calls=[] tool_call_chunks=[]
content=' Language' additional_kwargs={} response_metadata={} id='lc_run--019c1631-647b-7570-9709-1e7f8f6ef985' tool_calls=[] invalid_tool_calls=[] tool_call_chunks=[]
content=' Models' additional_kwargs={} response_metadata={} id='lc_run--019c1631-647b-7570-9709-1e7f8f6ef985' tool_calls=[] invalid_tool_calls=[] tool_call_chunks=[]
content=' (' additional_kwargs={} response_metadata={} id='lc_run--019c1631-647b-7570-9709-1e7f8f6ef985' tool_calls=[] invalid_tool_calls=[] tool_call_chunks=[]
content='LL' additional_kwargs={} response_metadata={} id='lc_run--019c1631-647b-7570-9